In [53]:
import pandas as pd
import numpy as np
import requests
import bs4 as bs
import urllib.request
import json
import requests
from tmdbv3api import TMDb
from tmdbv3api import Movie
import warnings
warnings.filterwarnings("ignore")

## Extracting features of 2020 movies from Wikipedia

In [4]:
link = "https://en.wikipedia.org/wiki/List_of_American_films_of_2020"

In [5]:
source=urllib.request.urlopen(link).read()

In [8]:
soup=bs.BeautifulSoup(source,"lxml")

In [13]:
tables=soup.find_all("table",class_="wikitable sortable")

In [15]:
len(tables)

4

In [17]:
type(tables[0])

bs4.element.Tag

In [18]:
type(tables[1])

bs4.element.Tag

In [27]:
df1=pd.read_html(str(tables[0]))[0]
df2=pd.read_html(str(tables[1]))[0]
df3=pd.read_html(str(tables[2]))[0]
df4=pd.read_html(str(tables[3]))[0]

In [28]:
df=df1.append(df2.append(df3.append(df4,ignore_index=True),ignore_index=True),ignore_index=True)

In [29]:
df.head()

,Opening,Opening.1,Title,Production company,Cast and crew,Ref.
0,JANUARY,3,The Grudge,Screen Gems / Stage 6 Films / Ghost House Pict...,Nicolas Pesce (director/screenplay); Andrea Ri...,[2]
1,JANUARY,10,Underwater,20th Century Fox / TSG Entertainment / Chernin...,"William Eubank (director); Brian Duffield, Ada...",[3]
2,JANUARY,10,Like a Boss,Paramount Pictures,"Miguel Arteta (director); Sam Pitman, Adam Col...",[4]
3,JANUARY,10,Three Christs,IFC Films,Jon Avnet (director/screenplay); Eric Nazarian...,NaN
4,JANUARY,10,Inherit the Viper,Barry Films / Tycor International Film Company,Anthony Jerjen (director); Andrew Crabtree (sc...,[5]


In [30]:
df_2020=df[["Title","Cast and crew"]]

In [31]:
df_2020.head(2)

,Title,Cast and crew
0,The Grudge,Nicolas Pesce (director/screenplay); Andrea Ri...
1,Underwater,"William Eubank (director); Brian Duffield, Ada..."


In [33]:
tmdb=TMDb()
tmdb.api_key="3cd610e5da6dcc2d11cb264184e57edb"

In [50]:
tmdb_movie=Movie()
def get_genres(x):
    gen=[]
    result=tmdb_movie.search(x)
    if not result:
        return np.NaN
    else:
        movie_id=result[0].id
        response=requests.get("https://api.themoviedb.org/3/movie/{}?api_key={}".format(movie_id,tmdb.api_key))
        data_json=response.json()
    if data_json["genres"]:
        genre_str=" "
        for i in range(0,len(data_json["genres"])):
            gen.append(data_json["genres"][i]["name"])
        return genre_str.join(gen)
    else:
        return np.NaN
    

In [54]:
df_2020["genres"]=df_2020["Title"].map(lambda x:get_genres(x))

In [55]:
df_2020.head()

,Title,Cast and crew,genres
0,The Grudge,Nicolas Pesce (director/screenplay); Andrea Ri...,Horror Mystery Thriller
1,Underwater,"William Eubank (director); Brian Duffield, Ada...",Action Horror Science Fiction Thriller
2,Like a Boss,"Miguel Arteta (director); Sam Pitman, Adam Col...",Comedy
3,Three Christs,Jon Avnet (director/screenplay); Eric Nazarian...,Drama
4,Inherit the Viper,Anthony Jerjen (director); Andrew Crabtree (sc...,Drama Thriller Crime


In [58]:
df_2020["Cast and crew"][1]

'William Eubank (director); Brian Duffield, Adam Cozad (screenplay); Kristen Stewart, Vincent Cassel, Jessica Henwick, John Gallagher Jr., Mamoudou Athie, T.J. Miller'

In [56]:
def get_director(x):
    if "(director)" in x:
        return x.split("(director)")[0]
    elif "(directors)" in x:
        return x.split("(directors)")[0]
    else:
        return x.split("(director/screenplay)")[0]

In [59]:
def get_actor1(x):
    if "(director/screenplay);" in x:
        return x.split("(director/screenplay);")[-1].split(",")[0]
    else:
        return x.split("(screenplay);")[-1].split(",")[0]
    

In [79]:
def get_actor2(x):
    if "(director/screenplay);" in x:
        if len(x.split("(director/screenplay);")[-1].split(","))>1:
            return x.split("(director/screenplay);")[-1].split(",")[1]
        else:
            return np.NaN
    else:
        if len(x.split("(screenplay);")[-1].split(","))>1:
            return x.split("(screenplay);")[-1].split(",")[1]
        else:
            return np.NaN    

In [80]:
def get_actor3(x):
    if "(director/screenplay);" in x:
        if len(x.split("(director/screenplay);")[-1].split(","))>2:
            return x.split("(director/screenplay);")[-1].split(",")[2]
        else:
            return np.NaN
    else:
        if len(x.split("(screenplay);")[-1].split(","))>2:
            return x.split("(screenplay);")[-1].split(",")[2]
        else:
            return np.NaN

In [81]:
df_2020["actor_1_name"]=df_2020["Cast and crew"].map(lambda x: get_actor1(x))

In [82]:
df_2020["actor_2_name"]=df_2020["Cast and crew"].map(lambda x: get_actor2(str(x)))

In [83]:
df_2020["actor_3_name"]=df_2020["Cast and crew"].map(lambda x: get_actor3(str(x)))

In [84]:
df_2020["director_name"]=df_2020["Cast and crew"].map(lambda x:get_director(x))

In [85]:
df_2020.head()

,Title,Cast and crew,genres,actor_1_name,actor_3_name,actor_2_name,director_name
0,The Grudge,Nicolas Pesce (director/screenplay); Andrea Ri...,Horror Mystery Thriller,Andrea Riseborough,John Cho,Demián Bichir,Nicolas Pesce
1,Underwater,"William Eubank (director); Brian Duffield, Ada...",Action Horror Science Fiction Thriller,Kristen Stewart,Jessica Henwick,Vincent Cassel,William Eubank
2,Like a Boss,"Miguel Arteta (director); Sam Pitman, Adam Col...",Comedy,Tiffany Haddish,Salma Hayek,Rose Byrne,Miguel Arteta
3,Three Christs,Jon Avnet (director/screenplay); Eric Nazarian...,Drama,Eric Nazarian (screenplay); Richard Gere,Walton Goggins,Peter Dinklage,Jon Avnet
4,Inherit the Viper,Anthony Jerjen (director); Andrew Crabtree (sc...,Drama Thriller Crime,Josh Hartnett,Chandler Riggs,Margarita Levieva,Anthony Jerjen


In [86]:
df_2020=df_2020.rename(columns={"Title":"movie_title"})

In [87]:
df_2020["movie_title"]=df_2020["movie_title"].str.lower()

In [90]:
new_df20 = df_2020.loc[:,['director_name','actor_1_name','actor_2_name','actor_3_name','genres','movie_title']]

In [91]:
new_df20['comb'] = new_df20['actor_1_name'] + ' ' + new_df20['actor_2_name'] + ' '+ new_df20['actor_3_name'] \
                   + ' '+ new_df20['director_name'] +' ' + new_df20['genres']

In [92]:
new_df20.isna().sum()

director_name     0
actor_1_name      0
actor_2_name      4
actor_3_name     27
genres            2
movie_title       0
comb             29
dtype: int64

In [96]:
new_df20.dropna(inplace=True)

In [97]:
old_df=pd.read_csv("final_data.csv")

In [98]:
final_df=old_df.append(new_df20,ignore_index=True)

In [100]:
final_df.to_csv("main_data.csv",index=False)

In [101]:
final_df.head()

,Unnamed: 0,director_name,actor_1_name,actor_2_name,actor_3_name,genres,movie_title,comb
0,0.0,James Cameron,CCH Pounder,Joel David Moore,Wes Studi,Action Adventure Fantasy Sci-Fi,avatar,CCH Pounder Joel David Moore Wes Studi James C...
1,1.0,Gore Verbinski,Johnny Depp,Orlando Bloom,Jack Davenport,Action Adventure Fantasy,pirates of the caribbean: at world's end,Johnny Depp Orlando Bloom Jack Davenport Gore ...
2,2.0,Sam Mendes,Christoph Waltz,Rory Kinnear,Stephanie Sigman,Action Adventure Thriller,spectre,Christoph Waltz Rory Kinnear Stephanie Sigman ...
3,3.0,Christopher Nolan,Tom Hardy,Christian Bale,Joseph Gordon-Levitt,Action Thriller,the dark knight rises,Tom Hardy Christian Bale Joseph Gordon-Levitt ...
4,4.0,Doug Walker,Doug Walker,Rob Walker,unknown,Documentary,star wars: episode vii - the force awakens ...,Doug Walker Rob Walker unknown Doug Walker Doc...
